In [1]:
import numpy as np
import copy
from imageio import imread
from scipy.spatial.distance import cdist

# Parameters
nrun = 20  # number of classification runs
fname_label = 'class_labels.txt'  # where class labels are stored for each run

In [2]:
ftype = 'cost'
folder = 'all_runs/run01/'
assert ((ftype == 'cost') | (ftype == 'score'))

In [3]:
# get file names
with open(folder + fname_label) as f:
    content = f.read().splitlines()
    
pairs = [line.split() for line in content]
test_files = [pair[0] for pair in pairs]
train_files = [pair[1] for pair in pairs]
answers_files = copy.copy(train_files)
test_files.sort()
train_files.sort()
ntrain = len(train_files)
ntest = len(test_files)

In [14]:
answers_files

['run01/training/class08.png',
 'run01/training/class09.png',
 'run01/training/class02.png',
 'run01/training/class19.png',
 'run01/training/class10.png',
 'run01/training/class18.png',
 'run01/training/class13.png',
 'run01/training/class01.png',
 'run01/training/class04.png',
 'run01/training/class11.png',
 'run01/training/class17.png',
 'run01/training/class03.png',
 'run01/training/class20.png',
 'run01/training/class07.png',
 'run01/training/class12.png',
 'run01/training/class05.png',
 'run01/training/class06.png',
 'run01/training/class15.png',
 'run01/training/class14.png',
 'run01/training/class16.png']

In [13]:
train_files

['run01/training/class01.png',
 'run01/training/class02.png',
 'run01/training/class03.png',
 'run01/training/class04.png',
 'run01/training/class05.png',
 'run01/training/class06.png',
 'run01/training/class07.png',
 'run01/training/class08.png',
 'run01/training/class09.png',
 'run01/training/class10.png',
 'run01/training/class11.png',
 'run01/training/class12.png',
 'run01/training/class13.png',
 'run01/training/class14.png',
 'run01/training/class15.png',
 'run01/training/class16.png',
 'run01/training/class17.png',
 'run01/training/class18.png',
 'run01/training/class19.png',
 'run01/training/class20.png']

In [6]:
def LoadImgAsPoints(fn):
    '''
    Load image file and return coordinates of 'inked' pixels in the binary image
    '''
    fn = 'all_runs/'+ fn
    I = imread(fn)
    I = np.array(I, dtype=bool)
    I = np.logical_not(I)
    (row, col) = I.nonzero()
    D = np.array([row, col]).T.astype(float)
    D = D - D.mean(axis=0)
    return D

def ModHausdorffDistance(itemA, itemB):
    # Modified Hausdorff Distance
    #
    # Input
    #  itemA : [n x 2] coordinates of "inked" pixels
    #  itemB : [m x 2] coordinates of "inked" pixels
    #
    #  M.-P. Dubuisson, A. K. Jain (1994). A modified hausdorff distance for object matching.
    #  International Conference on Pattern Recognition, pp. 566-568.
    #
    D = cdist(itemA, itemB)
    mindist_A = D.min(axis=1)
    mindist_B = D.min(axis=0)
    mean_A = np.mean(mindist_A)
    mean_B = np.mean(mindist_B)
    return max(mean_A, mean_B)

In [7]:
# load the images (and, if needed, extract features)
train_items = [LoadImgAsPoints(f) for f in train_files]
test_items = [LoadImgAsPoints(f) for f in test_files]

In [10]:
# compute cost matrix
costM = np.zeros((ntest, ntrain), float)
for i in range(ntest):
    for c in range(ntrain):
        costM[i, c] = ModHausdorffDistance(test_items[i], train_items[c])

In [11]:
if ftype == 'cost':
    YHAT = np.argmin(costM, axis=1)
elif ftype == 'score':
    YHAT = np.argmax(costM, axis=1)
else:
    assert False

# compute the error rate
correct = 0.0
for i in range(ntest):
    if train_files[YHAT[i]] == answers_files[i]:
        correct += 1.0
pcorrect = 100 * correct / ntest
perror = 100 - pcorrect
perror